# Deployment Insights: Access via Trino

The easy way to access the deployment access database is through this notebook that connects to the Trino instance hosted in AWS. Please update the first cell with appropriate config details.

# Trino Setup
For the SQL queries to exexute in this notebook, you must have a Trino server running, connected to the Aerospike database via the Aerospike Trino Connector. 

Set the following Trino server parameters.

In [2]:
TRINO_IP = "<IP>"
TRINO_PORT = "<port>"
TRINO_USER = "<user>"
TRINO_PASSWORD = "<password>"

## Trino Command
Define the environment variable for short form of the Trino command. 

You can also run the Trino command line tool in a separate shell tab.

In [3]:
%env TRINO=../trino --server $TRINO_IP:$TRINO_PORT --catalog aerospike --schema test --output-format=TSV_HEADER
%env TRINO_VERTICAL=../trino --server $TRINO_IP:$TRINO_PORT --catalog aerospike --schema test --output-format=VERTICAL

env: TRINO=../trino --server 184.169.149.193:8080 --catalog aerospike --schema test --output-format=TSV_HEADER
env: TRINO_VERTICAL=../trino --server 184.169.149.193:8080 --catalog aerospike --schema test --output-format=VERTICAL


# Trino SQL Queries

Show schemas (namespaces).

In [4]:
!$TRINO --execute "show schemas";

Schema
bar
information_schema
test


Show tables (sets).

In [5]:
!$TRINO --execute "show tables";

Table
__default
insights


Show a sample record.

In [6]:
!$TRINO_VERTICAL --execute "select * from test.insights limit 1" ;


-[ RECORD 1 ]---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
__key           | NULL
case_num        | 25196
cluster_name    | 10.240.0.54:3000
cluster_size    | 5
customer        | Sovrn Holdings
features        | ["xdr_dest","scan","kvs","batch"]
namespaces      | [{"num_sets":17,"replication_factor":"2","num_sindex":0,"num_bins":"157","objects":"2191560712","single_bin":"false","master_objects":"1101515628","strong_consistency":"false","data_in_index":"false","enable_xdr":"false","name":"addelivery","storage_engine":"device","ns_cluster_size":"5"}]
server_release  | 4.8.0.4
storage_engines | ["device"]
timestamp       | 2021-10-12 16:31:30 UTC
total_objects   | 2191560712


Get customers using feature 'index_on_device'.
```
select customer, cluster_name, features 
from insights 
where contains(cast(features as array(VARCHAR)),'index_on_device')
```

In [7]:
!$TRINO --execute "select customer, cluster_name, features from insights where contains(cast(features as array(VARCHAR)),'index_on_device')" ;


customer	cluster_name	features
AppsFlyer	aerospike-clicks-pointers-037-eu1	["batch","kvs","security","index_on_device"]
Charles Schwab	TIGER-QA2	["xdr_src","tls_service","index_on_device","security","tls_fabric","sc","sindex","rack_aware","batch","kvs"]
Verizon Media (Dublin Ireland)	tgt-platform.triton.aerospike	["tls_service","index_on_device","kvs","scan","tls_fabric"]
AdTalos  - 新义互联科技有限公司	xapi	["index_on_device","udf","kvs","batch"]
Charles Schwab	TIGER-QA1	["sindex","scan","tls_service","rack_aware","batch","xdr_dest","index_on_device","kvs","query","tls_fabric"]
Neustar, Inc. - San Diego	10.103.25.5:3000	["index_on_device","kvs","batch","security","xdr_dest"]
Charles Schwab	TIGER-QA2	["tls_service","index_on_device","xdr_dest","tls_fabric","sindex","kvs","batch","rack_aware","scan","query"]
Charles Schwab	TIGER-PDC1	["sindex","scan","sc","tls_service","rack_aware","batch","security","xdr_dest","index_on_device","kvs","query","tls_fabric"]
Quantcast Corporation	10.89.32.254:3000	

Get customers using feature 'xdr_dest' and release after 5.x
```
select customer, features, server_release 
from insights 
where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*')
```

In [8]:
!$TRINO --execute "select customer, features, server_release from insights where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*');" ;


customer	features	server_release
LexisNexis Risk Solutions Inc. (ThreatMetrix)	["xdr_src","batch","rack_aware","kvs","security","xdr_dest","udf","index_on_pmem"]	5.7.0.7
Bharti Airtel Limited	["batch","scan","kvs","query","udf","xdr_dest"]	5.5.0.12
Neustar, Inc. - San Diego	["security","xdr_dest","index_on_device","kvs"]	5.6.0.8
Charles Schwab	["sindex","scan","sc","tls_service","rack_aware","batch","security","xdr_dest","index_on_device","kvs","query","tls_fabric"]	5.2.0.21
DoubleVerify	["scan","kvs","udf","xdr_dest"]	5.2.0.6
Charles Schwab	["sindex","scan","sc","tls_service","rack_aware","batch","security","xdr_dest","index_on_device","kvs","query","tls_fabric"]	5.2.0.21
Wayfair LLC	["batch","rack_aware","kvs","security","xdr_dest","scan","udf"]	5.5.0.3
Verizon Media	["tls_service","xdr_dest","security","tls_fabric","index_on_pmem","sc","rack_aware","batch","kvs","scan"]	5.6.0.13
Verizon Media	["index_on_pmem","scan","sc","tls_service","rack_aware","batch","security","xdr_dest","kvs"

Get the largest deployed cluster by each customer.
```
select customer, max(cluster_size) as max_cluster_size
rom insights
group by customer
```

In [9]:
!$TRINO --execute "select customer, max(cluster_size) as max_cluster_size from insights group by customer" ;


customer	max_cluster_size
Hoonar Tekworks Consulting LLP	4
Five9	3
Choreograph, LLC	3
Passendo ApS	12
Connatix	2
Adobe Systems Incorporated	12
Organic Response	1
Barclays PLC	78
VPBank SMBC Finance Company Limited	4
Beijing Qiyuan Technology Co. Ltd	4
Clinch Labs Ltd	18
DoubleVerify	12
PubMatic	8
Verizon Media	104
Cybereason Inc	22
Viant - Specific Media /Myspace/Xumo	8
Supership, Inc.	5
Amdocs Development Limited (Channels Account)	18
Roku DX Holdings Inc.	20
Beeswax	20
Ping An Bank	7
ZIR AI	3
Sovrn Holdings	5
Starkware Industries Ltd	31
Affle International Pte Ltd	14
Playtika	7
Airtel - MyNetx	4
Quantcast Corporation	28
Bombinate Technologies Pvt Ltd.	16
Tanla Solutions Limited	1
SAS (Channels Account)	1
DeepIntent, Inc.	12
PT GoTo Gojek Tokopedia	12
AppsFlyer	110
Experian Services Corp.	3
Banca D'Italia	6
Caulis Inc.	14
AdTalos  - 新义互联科技有限公司	10
Unity Technologies	12
Dailyhunt	5
PayPal, Inc.	30
Adjust GmbH	46
YouAppi INC	3
Xandr	3
DBS Bank Ltd	5
Mparticle Inc.	12
Bharti Airtel Limite

Get the largest namespace by objects for each customer.
```
select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) 
                 as max_ns_objects
from insights 
group by customer
```

In [10]:
!$TRINO --execute "select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) as max_ns_objects from insights group by customer" ;


customer	max_ns_objects
Adobe Systems Incorporated	[680914382, 0]
Organic Response	[4300647, 119548, 602544, 2446359, 47689141, 359198]
Barclays PLC	[682387527]
Beijing Qiyuan Technology Co. Ltd	[197067431]
VPBank SMBC Finance Company Limited	[23839637, 74399476]
Hoonar Tekworks Consulting LLP	[2469048553]
Five9	[31547069, 0]
Choreograph, LLC	[2538292538]
Connatix	[1016585625]
Passendo ApS	[68659919, 1353548]
Clinch Labs Ltd	[76278262]
DoubleVerify	[294938830, 0]
PubMatic	[620377899, 613308, 722236616, 23679]
Verizon Media	[88612072, 5, 109, 1212073, 719238568, 546414097, 635967749, 13349290, 0, 833314383, 10607]
Cybereason Inc	[871553964, 1212]
Supership, Inc.	[46233734]
Viant - Specific Media /Myspace/Xumo	[10546224]
LexisNexis Risk Solutions Inc. (ThreatMetrix)	[9202388756]
Aerospike Cloud Managed Service	[119, 111655]
Phonepe Private Limited (FX Mart Pvt. Ltd.)	[6835608, 121806857]
Deqwas, Inc.	[857023949]
Federal Reserve Bank of Boston	[0]
Reliance Industries Limited	[730042, 7295

Get customers with single-bin namespaces. Also get (name, storage, data_in_index) for each single-bin namespace.

`
select customer, case_num, single_bin_ns from (
    select customer, transform(
        filter(cast(namespaces as array<map<varchar,varchar>>), 
               ns->ns['single_bin']='true'),
        ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) 
        ) as single_bin_ns 
    from insights)
where single_bin_ns != Array[]
order by customer, case_num desc;
`

In [11]:
!$TRINO_VERTICAL --execute "select customer, case_num, cluster_name, single_bin_ns from (select customer, case_num,  cluster_name, transform(filter(cast(namespaces as array<map<varchar,varchar>>), ns->ns['single_bin']='true'),ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) ) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc;"


-[ RECORD 1 ]-+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
customer      | AdTalos  - 新义互联科技有限公司
case_num      | 25691
cluster_name  | xapi
single_bin_ns | [{device-set, device, false}, {mapping, device, false}, {frequency, device, true}]
-[ RECORD 2 ]-+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
customer      | AdTalos  - 新义互联科技有限公司
case_num      | 25106
cluster_name  | xapi
single_bin_ns | [{device-set, device, false}, {mapping, device, false}, {frequency, device, true}]
-[ RECORD 3 ]-+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Get customers that have single-bin in-memory namespaces.

`
select customer, case_num, cluster_name, single_bin_mem_ns from (
    select customer,  case_num, cluster_name, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), 
                   ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' 
                                                    or ns['data_in_index'] = 'true'))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [12]:
!$TRINO --execute "select customer, case_num, cluster_name, single_bin_mem_ns from (select customer, case_num, cluster_name, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or  ns['data_in_index'] = 'true')) as single_bin_mem_ns from insights) where single_bin_mem_ns != Array[] order by customer, case_num desc";


customer	case_num	cluster_name	single_bin_mem_ns
AdTalos  - 新义互联科技有限公司	25691	xapi	[{data_in_index=true, name=frequency, single_bin=true, storage_engine=device}]
AdTalos  - 新义互联科技有限公司	25106	xapi	[{data_in_index=true, name=frequency, single_bin=true, storage_engine=device}]
Compulse, LLC	24833	10.0.1.213:3000	[{data_in_index=true, enable_xdr=false, name=impression_cap, single_bin=true, storage_engine=device}]
Compulse, LLC	24568	10.0.1.213:3000	[{data_in_index=true, enable_xdr=false, name=impression_cap, single_bin=true, storage_engine=device}]
InMarket Media LLC	24444	aerospike-us-east-1e-tl-v9-stack	[{data_in_index=false, enable_xdr=false, name=tiles, single_bin=true, storage_engine=memory}]
Quantcast Corporation	26029	10.89.32.254:3000	[{data_in_index=false, name=in-memory-rtff, single_bin=true, storage_engine=memory}]
Quantcast Corporation	24960	10.91.34.195:3000	[{data_in_index=false, name=in-memory-rtff, single_bin=true, storage_engine=memory}]
Sporta Technologies Private Limited (

Get customers that have single-bin in-memory namespaces, with xdr_src flag to indicate use of XDR.

`
select customer, case_num, cluster_name, xdr_src, single_bin_mem_ns from (
    select customer, case_num, cluster_name, contains(cast(features as array<varchar>), 
                                                     'xdr_src') as xdr_src, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','enable_xdr','data_in_index))), 
                    ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or 
                                                     ns['data_in_index'] = 'true' ))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [13]:
!$TRINO --execute "select case_num, customer, cluster_name, xdr_src, single_bin_ns from (select case_num, customer, cluster_name, contains(cast(features as array<varchar>), 'xdr_src') as xdr_src, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','enable_xdr','data_in_index'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or ns['data_in_index'] = 'true')) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc";


case_num	customer	cluster_name	xdr_src	single_bin_ns
25691	AdTalos  - 新义互联科技有限公司	xapi	false	[{data_in_index=true, name=frequency, single_bin=true, storage_engine=device}]
25106	AdTalos  - 新义互联科技有限公司	xapi	false	[{data_in_index=true, name=frequency, single_bin=true, storage_engine=device}]
24833	Compulse, LLC	10.0.1.213:3000	true	[{data_in_index=true, enable_xdr=false, name=impression_cap, single_bin=true, storage_engine=device}]
24568	Compulse, LLC	10.0.1.213:3000	true	[{data_in_index=true, enable_xdr=false, name=impression_cap, single_bin=true, storage_engine=device}]
24444	InMarket Media LLC	aerospike-us-east-1e-tl-v9-stack	false	[{data_in_index=false, enable_xdr=false, name=tiles, single_bin=true, storage_engine=memory}]
26029	Quantcast Corporation	10.89.32.254:3000	false	[{data_in_index=false, name=in-memory-rtff, single_bin=true, storage_engine=memory}]
24960	Quantcast Corporation	10.91.34.195:3000	false	[{data_in_index=false, name=in-memory-rtff, single_bin=true, storage_engine=me

Get features used in each deployed cluster.

In [15]:
!$TRINO --execute "select customer, cluster_name, max(timestamp) as last, features from insights group by customer, cluster_name, timestamp, features order by customer, cluster_name;"


customer	cluster_name	last	features
AcuityAds	10.64.97.103:3000	2021-05-27 17:26:41 UTC	["scan","kvs","xdr_src","xdr_dest"]
AdDaptive Intelligence	10.100.0.206:3000	2021-07-27 12:26:32 UTC	["err_format"]
AdDaptive Intelligence	10.100.1.88:3000	2021-07-27 12:26:50 UTC	["scan","kvs"]
AdTalos  - 新义互联科技有限公司	xapi	2021-12-08 10:23:37 UTC	["index_on_device","kvs","batch","scan","udf"]
AdTalos  - 新义互联科技有限公司	xapi	2021-09-30 00:56:14 UTC	["index_on_device","udf","kvs","batch"]
Adition Technologies GmbH	172.20.0.169:3000	2021-09-08 04:23:31 UTC	["batch","kvs"]
Adition Technologies GmbH	172.20.0.18:3000	2021-09-08 04:19:50 UTC	["batch","kvs"]
Adition Technologies GmbH	172.20.0.58:3000	2021-09-08 04:21:44 UTC	["batch","kvs"]
Adition Technologies GmbH	172.20.0.97:3000	2021-09-08 04:25:04 UTC	["batch","kvs","query","scan"]
Adition Technologies GmbH	172.20.1.171:3000	2021-09-08 04:29:08 UTC	["batch","kvs","query","scan"]
Adition Technologies GmbH	172.20.1.179:3000	2021-09-08 04:28:38 UTC	["kvs"]
Aditi

# Programmatic Access via Trino
Install the trino package if not already installed to enable Python access by running the following cell.

In [17]:
#!pip install trino


In [18]:
import trino
conn = trino.dbapi.connect(
    host=TRINO_IP,
    port=TRINO_PORT,
    user='admin',
    catalog='aerospike',
    schema='test'
)
cur = conn.cursor()
cur.execute('select * from insights limit 1')
rows = cur.fetchall()
print(rows)

[[None, 25707, 'aerospike-atlas_users-dedi_us-east-1_prod', 99, 'Snapchat (Snap, Inc.)', '["batch","tls_service","rack_aware","kvs","tls_fabric","sc"]', '[{"num_sets":8,"replication_factor":"3","num_sindex":0,"data_in_index":"false","num_bins":"1","objects":"361658916","name":"memory","single_bin":"false","ns_cluster_size":"99","storage_engine":"memory","master_objects":"123117143","strong_consistency":"true"}]', '5.5.0.9', '["memory"]', '2021-12-11 03:42:38 UTC', 361658916]]
